# explore gold and silver using FRED data

In [ ]:
library(tidyverse)
library(odbc)
library(plutoR)
options("scipen"=999)
source("config.R")
fred<-Fred()

In [ ]:
goldFred <- fred$Meta() %>% inner_join(fred$TimeSeries(), by = 'SERIES_ID') %>%
    filter(NAME %like% '%gold%') %>%
    group_by(SERIES_ID, TICKER, NAME) %>%
    summarize(MIN_TS = min(TIME_STAMP), MAX_TS = max(TIME_STAMP)) %>%
    select(SERIES_ID, TICKER, NAME, MIN_TS, MAX_TS)

print(data.frame(goldFred))

In [ ]:
silverFred <- fred$Meta() %>% inner_join(fred$TimeSeries(), by = 'SERIES_ID') %>%
    filter(NAME %like% '%silver%') %>%
    group_by(SERIES_ID, TICKER, NAME) %>%
    summarize(MIN_TS = min(TIME_STAMP), MAX_TS = max(TIME_STAMP)) %>%
    select(SERIES_ID, TICKER, NAME, MIN_TS, MAX_TS)

print(data.frame(silverFred))

it looks like FRED only has data on silver from 2017. So lets focus on gold for now.

In [ ]:
goldDf<-data.frame(fred$TimeSeries() %>% filter(SERIES_ID == -2147252004, TIME_STAMP >= as.Date('1995-01-01')) %>% select(TIME_STAMP, VAL))
print(nrows(goldDf))
print(head(goldDf))